In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import keras
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pickle
import numpy as np
import cv2
from numpy.random import default_rng
from tensorflow.keras.applications.efficientnet import EfficientNetB0, EfficientNetB3, EfficientNetB7
from keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import decode_predictions
from keras.layers import Input,Flatten,Dense,Activation
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt # Edited

In [ ]:
file = open(r"/content/drive/MyDrive/dataset/dataset.pkl", "rb") # Edited
data = pickle.load(file)
file.close()

inmeYok = data['inmeYok']
iskemi = data['iskemi']
kanama = data['kanama']
del data
data = None

inmeVar = np.concatenate((iskemi,kanama))
del iskemi
del kanama
iskemi = None
kananma = None

inmeYok_Y = np.zeros((len(inmeYok)))
inmeVar_Y = np.ones((len(inmeVar)))

In [ ]:
inmeVarTrain_X , inmeVarTrain_Y , inmeVarTest_X , inmeVarTest_Y = train_test_split(inmeVar,inmeVar_Y,test_size = 0.15,random_state = 42)
del inmeVar
inmeVar = None


inmeYokTrain_X , inmeYokTrain_Y , inmeYokTest_X , inmeYokTest_Y = train_test_split(inmeYok,inmeYok_Y,test_size = 0.15,random_state = 42)
del inmeYok
inmeYok = None

print(str(len(inmeYokTrain_X)) + " " + str(len(inmeVarTrain_X)))

rng = default_rng()
list_inmeYok = rng.choice(len(inmeYokTrain_X)-1, size=len(inmeVarTrain_X), replace=False)
select_no_hemoraj = []
for i in range(len(inmeVarTrain_X)):
    select_no_hemoraj.append(inmeYokTrain_X[list_inmeYok[i]])


del inmeYokTrain_X
inmeYokTrain_X = None

train_x = np.concatenate((np.array(select_no_hemoraj),inmeVarTrain_X),0)
train_y = np.concatenate((np.zeros(len(select_no_hemoraj)),np.ones(len(inmeVarTrain_X))))
del select_no_hemoraj
del inmeVarTrain_X
select_no_hemoraj = None
inmeVarTrain_X = None


In [ ]:
split = 30 # Yüzde # Edited
rstate = 43 # Edited

train_x,test_x,train_y,test_y = train_test_split(train_x, train_y, test_size = split/100, random_state = rstate) # Edited

train_yy = []
for i in range(len(train_y)):
    if(train_y[i] == 0):
        train_yy.append([1,0])
    else:
        train_yy.append([0,1])
train_y = np.array(train_yy)

test_yy = []
for i in range(len(test_y)):
    if(test_y[i] == 0):
        test_yy.append([1,0])
    else:
        test_yy.append([0,1])
test_y = np.array(test_yy)

In [ ]:
### ÇALIŞILACAK MODELIN , KATMAN SAYILARININ , KATMAN BOYUTLARININ ve EGITILMEYECEK KATMANLARIN BELIRLENDIGI YER ###

model = DenseNet121(include_top=False, input_shape=(512, 512, 3)) # Edited
for layer in model.layers: 
    layer.trainable = False
flat1 = Flatten()(model.layers[-1].output)
#class1 = Dense(1024, activation='relu')(flat1)
output = Dense(2, activation='softmax')(flat1)

model = Model(inputs=model.inputs, outputs=output)
model.summary()

In [ ]:
### MODELİN PARAMETRELERİNİN BELİRLENİP , EĞİTİM YAPTIKTAN SONRA TEST YAPILDIĞI YER ###
# Cell Edited
ep = 30
bs = 32
lr = 1 # Yüzde

opt = keras.optimizers.Adam(learning_rate=lr/100)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs = ep, batch_size = bs , verbose = 1, validation_data = (test_x, test_y))

path = "/content/drive/MyDrive/DenseNet/121/" + "Ep=" + str(ep) + "-Batch=" + str(bs) + "-LRate=%" + str(lr) + "-Rndm=" + str(rstate) + "-TestSze=%" + str(split) + "-Acc=" + str(history.history['val_accuracy'][-1])
model.save(path)
model.save(path + "/mymodel.h5") # .h5 olarak da kaydettim

In [ ]:
# Edited
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train Acc', 'Validation Acc'], loc='lower right')
plt.savefig(path + "/Graph_Acc.png")
plt.show()

In [ ]:
# Edited
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train Loss', 'Validation Loss'], loc='upper right')
plt.savefig(path + "/Graph_Loss.png")
plt.show()

In [ ]:
# Rastgele tahmin
predict = model.predict(test_x[0:10])
print(predict)
print(test_y[0:10])

In [ ]:
# Modeli yeniden yükleyip test etme
from keras.models import load_model
newmodel = load_model(path + "/mymodel.h5")
newpredict = newmodel.predict(test_x[0:20])
print(newpredict)
print(test_y[0:20])